In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import folium
import warnings

warnings.simplefilter('ignore')
plt.style.use('ggplot')

# MARTINI - Danish river data

Initial exploration of the newly released "ODA for alle" database for Danish water chemistry ([here](https://odaforalle.au.dk)).

## 1. Stations of interest

André has previously sent an Excel file listing MARTINI river outlets.

In [2]:
# Read Andre's list of rivers
riv_xlsx = r'../tidied_data_series/martini_river_outlets.xlsx'
riv_df = pd.read_excel(riv_xlsx, sheet_name='rivers')

# Get Danish rivers
riv_df = riv_df[riv_df['martini_code'].str.startswith('dmca')]

riv_df

,martini_id,station_name,resa_id,vannmiljo_id,mvm_id,lat,lon,martini_code
0,34,Vigsø bukt,NaN,NaN,NaN,57.099800,8.757300,dmca003
1,35,West side of Nord-Jylland - Liver Å,NaN,NaN,NaN,57.533000,9.886600,dmca004
2,36,Hirtshals-Skagen - Uggerby Å,NaN,NaN,NaN,57.592200,10.149800,dmca005
3,37,Skagen- Lyngsaa strand,NaN,NaN,NaN,57.595285,10.170326,dmca006
4,38,East side of Nord-Jylland,NaN,NaN,NaN,57.653900,10.479300,dmca007
5,39,Limfjord,NaN,NaN,NaN,56.976100,10.332000,dmca008
6,40,Mariagerfjord,NaN,NaN,NaN,56.711400,10.353100,dmca009
7,41,Gudenå,NaN,NaN,NaN,56.602300,10.346300,dmca010
8,42,Læsø,NaN,NaN,NaN,57.281700,11.149100,dmca011


In [3]:
# Map
nivapy.spatial.quickmap(riv_df, 
                        lat_col='lat', 
                        lon_col='lon', 
                        popup='martini_code',
                        tiles='openstreetmap')

Some points to note based on exploration of the ODA database:

 * There doesn't appear to be any major surface drainage into Visgø Bugt (`dmca003`). As far as I can tell, most streams in this area actually flow southwards into Limfjorden. Perhaps this site should be removed?
 
 * There is no monitoring data at all for the island of Læsø (`dmca011`)
 
 * The three stations connected to Aalborg Bugt (`dmca008` to `dmca010`) are at the mouths of major estuaries. ODA classifies these as "marine" and water chemistry data are available, but not discharge. This water chemistry will not be representative of river inputs, though. The best I can do is  extract data for the main tributaries upstream, but I'm not sure how easy it will be to connect these to the MARTINI model?
 
 * ODA shows several major rivers along the northern part of the east coast, around Frederikshavn. These are not included in André's dataset
 
 * I don't think it will be possible to get a good overview of inputs/fluxes from Limfjorden, as it's not clear what flows where
 
## 2. Extract discharge and water chemistry

I've used ODA to try to find as many stations as possible with both discharge and water chemistry between 2000 and the present day.

In [4]:
# Read Q
q_xlsx = r'../../../oda_export/oda_danish_q.xlsx'
q_df = pd.read_excel(q_xlsx, sheet_name='Hent data')
q_df.head()

,Lokalitetsnavn,ObservationsStedNr,ObservationsStedNavn,Dato,Xutm_Euref89_Zone32,Yutm_Euref89_Zone32,Parameter,Enhed,Resultat,Kontrol,Kvalitet
0,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",20170101,543778,6359267,Vandføring,m3/s,0.341507,FagK,GODK
1,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",20170102,543778,6359267,Vandføring,m3/s,0.294721,FagK,GODK
2,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",20170103,543778,6359267,Vandføring,m3/s,0.274884,FagK,GODK
3,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",20170104,543778,6359267,Vandføring,m3/s,0.291034,FagK,GODK
4,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",20170105,543778,6359267,Vandføring,m3/s,0.240595,FagK,GODK


In [5]:
# Get unique Q stns
q_stns = q_df[['Lokalitetsnavn', 'ObservationsStedNr', 
               'ObservationsStedNavn', 'Xutm_Euref89_Zone32', 
               'Yutm_Euref89_Zone32']].drop_duplicates().reset_index(drop=True)

# Convert co-ords
q_stns['utm_zone'] = 32
q_stns = nivapy.spatial.utm_to_wgs84_dd(q_stns,
                                        east='Xutm_Euref89_Zone32',
                                        north='Yutm_Euref89_Zone32',
                                        zone='utm_zone')

del q_stns['Xutm_Euref89_Zone32'], q_stns['Yutm_Euref89_Zone32'], q_stns['utm_zone']
q_stns['station_name'] = q_stns['ObservationsStedNavn']

q_stns

,Lokalitetsnavn,ObservationsStedNr,ObservationsStedNavn,lat,lon,station_name
0,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",57.374123,9.727980,"Klostergrøften, Klostergrøften."
1,ELLING Å,2000005,"ELLING Å, ELLING KIRKE",57.472374,10.485345,"ELLING Å, ELLING KIRKE"
2,SÆBY Å,2000006,"SÆBY Å, HUMMELBRO",57.338617,10.487633,"SÆBY Å, HUMMELBRO"
3,KNASBORG Å,2000014,"KNASBORG Å, V. HOVEDVEJEN",57.582613,10.412861,"KNASBORG Å, V. HOVEDVEJEN"
4,UGGERBY Å,3000002,"UGGERBY Å, NS RANSBÆK",57.569108,10.108817,"UGGERBY Å, NS RANSBÆK"
5,LIVER Å,4000005,"LIVER Å, RØDE BRO",57.515902,9.898408,"LIVER Å, RØDE BRO"
6,VOER Å,5000003,"VOER Å, FÆBROEN",57.208349,10.437869,"VOER Å, FÆBROEN"
7,GERÅ,8000001,"GERÅ, MELHOLT KIRKE",57.103715,10.351597,"GERÅ, MELHOLT KIRKE"
8,KASTBJERG Å,15000002,"KASTBJERG Å, NORUP",56.676218,10.103505,"KASTBJERG Å, NORUP"
9,HASLEVGÅRDS Å,15000032,"HASLEVGÅRDS Å, TRÆPÆLEBRO",56.795681,10.228893,"HASLEVGÅRDS Å, TRÆPÆLEBRO"


In [6]:
# Read chem
wc_xlsx = r'../../../oda_export/oda_danish_chem.xlsx'
wc_df = pd.read_excel(wc_xlsx, sheet_name='Hent data')
wc_df.head()

,Lokalitetsnavn,ObservationsStedNr,ObservationsStedNavn,Xutm_Euref89_Zone32,Yutm_Euref89_Zone32,MC-stationsnr,Startdato,Startklok,Prøvetype,Udstyr,Parameter,Enhed,Prøvefraktion,Resultat,ResultatAttribut,Kontrol,Kvalitet,Oplukningskorrigeret
0,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",543778,6359267,NOR1000315,20161212,945,Enkeltprøve,Flaske,"Phosphor, total-P",mg/l,Total,0.095712,NaN,FagK,GODK,J
1,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",543778,6359267,NOR1000315,20161212,945,Enkeltprøve,Flaske,"Nitrogen,total N",mg/l,Total,4.582832,NaN,FagK,GODK,J
2,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",543778,6359267,NOR1000315,20161212,945,Enkeltprøve,Flaske,Suspenderede stoffer,mg/l,Total,10.000000,NaN,FagK,GODK,N
3,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",543778,6359267,NOR1000315,20161212,945,Enkeltprøve,Flaske,Ammoniak+ammonium-N,mg/l,Opløst - filtrat fra filtrering,0.150000,NaN,FagK,GODK,N
4,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",543778,6359267,NOR1000315,20161212,945,Enkeltprøve,Flaske,Ortho-phosphat-P,mg/l,Opløst - filtrat fra filtrering,0.027000,NaN,FagK,GODK,N


In [7]:
# Get unique chem stns
wc_stns = wc_df[['Lokalitetsnavn', 'ObservationsStedNr', 
                 'ObservationsStedNavn', 'Xutm_Euref89_Zone32', 
                 'Yutm_Euref89_Zone32']].drop_duplicates().reset_index(drop=True)

# Convert co-ords
wc_stns['utm_zone'] = 32
wc_stns = nivapy.spatial.utm_to_wgs84_dd(wc_stns,
                                         east='Xutm_Euref89_Zone32',
                                         north='Yutm_Euref89_Zone32',
                                         zone='utm_zone')

del wc_stns['Xutm_Euref89_Zone32'], wc_stns['Yutm_Euref89_Zone32'], wc_stns['utm_zone']
wc_stns['station_name'] = wc_stns['ObservationsStedNavn']

wc_stns

,Lokalitetsnavn,ObservationsStedNr,ObservationsStedNavn,lat,lon,station_name
0,Klostergrøften,1000315,"Klostergrøften, Klostergrøften.",57.374123,9.727980,"Klostergrøften, Klostergrøften."
1,ELLING Å,2000005,"ELLING Å, ELLING KIRKE",57.472374,10.485345,"ELLING Å, ELLING KIRKE"
2,SÆBY Å,2000006,"SÆBY Å, HUMMELBRO",57.338617,10.487633,"SÆBY Å, HUMMELBRO"
3,KNASBORG Å,2000014,"KNASBORG Å, V. HOVEDVEJEN",57.582613,10.412861,"KNASBORG Å, V. HOVEDVEJEN"
4,UGGERBY Å,3000002,"UGGERBY Å, NS RANSBÆK",57.569108,10.108817,"UGGERBY Å, NS RANSBÆK"
5,LIVER Å,4000005,"LIVER Å, RØDE BRO",57.515902,9.898408,"LIVER Å, RØDE BRO"
6,VOER Å,5000003,"VOER Å, FÆBROEN",57.208349,10.437869,"VOER Å, FÆBROEN"
7,GERÅ,8000001,"GERÅ, MELHOLT KIRKE",57.103715,10.351597,"GERÅ, MELHOLT KIRKE"
8,KASTBJERG Å,15000002,"KASTBJERG Å, NORUP",56.676218,10.103505,"KASTBJERG Å, NORUP"
9,HASLEVGÅRDS Å,15000032,"HASLEVGÅRDS Å, TRÆPÆLEBRO",56.795681,10.228893,"HASLEVGÅRDS Å, TRÆPÆLEBRO"


All 19 stations selected have both discharge and (some) chemistry data. However, in order to be used for MARTINI, we *at least* need discharge for the period from 01/01/2015 to 31/12/2017.

In [8]:
# Get stns with Q measured during period of interest
q_df['Dato'] = pd.to_datetime(q_df['Dato'], format='%Y%m%d')
agg_df = q_df.groupby('ObservationsStedNr')['Dato'].agg(['min', 'max'])
condition = (agg_df['min'] <= '2015-01-01') & (agg_df['max'] >= '2017-12-31')
agg_df = agg_df[condition]
print('%s stations have discharge spanning 2015 - 2017' % len(agg_df))
stns = agg_df.index.values

agg_df

12 stations have discharge spanning 2015 - 2017


,min,max
ObservationsStedNr,,
2000005,2000-01-01,2018-12-31
2000006,2000-01-01,2018-12-31
3000002,2000-01-01,2018-12-31
4000005,2000-01-01,2018-12-31
5000003,2000-01-01,2018-12-31
8000001,2000-01-01,2018-12-31
15000002,2000-01-01,2018-12-31
15000032,2000-01-01,2018-12-31
15000034,2000-01-01,2018-12-31


In [9]:
# Get just relevant stations
q_stns = q_stns.query('ObservationsStedNr in @stns')
wc_stns = wc_stns.query('ObservationsStedNr in @stns')

In [12]:
# Setup map
map1 = folium.Map(location=[57, 10.8],
                  zoom_start=8,
                  tiles='openstreetmap')

# Create feature groups
fg_a = folium.FeatureGroup(name='Andre river outlets')
fg_qwc = folium.FeatureGroup(name='Discharge and chemistry stations')
fgs = [fg_a, fg_qwc]

# Add clickable markers for sites
cols = ['red', 'green']
for df_idx, df in enumerate([riv_df, q_stns]):
    for idx, row in df.iterrows():  
        folium.Marker([row['lat'], row['lon']], 
                      popup=row['station_name'],
                      icon=folium.Icon(color=cols[df_idx])).add_to(fgs[df_idx])
    map1.add_child(fgs[df_idx])

# Turn on layer control
map1.add_child(folium.map.LayerControl())

map1